In [2]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

In [17]:
# Metodo de Jacobi:  diagonalizacion de matrices simetrica

# Matriz de rotacion
def R(i, j, t, n):
    
    R = np.eye(n)
    R[i, i] = np.cos(t)
    R[j, j] = np.cos(t)
    R[i, j] = -np.sin(t)
    R[j, i] = np.sin(t)
    return R

In [18]:
def get_greatest_element(A):
    a = 0 
    I, J = 0, 0
    n = np.shape(A)[0]
    for i in range(n):
        for j in range(i+1, n):
            if np.abs(A[i,j]) > a:
                I, J = i, j
                a = np.abs(A[i,j])
    return I, J, a


In [26]:
# Funcion que diagonaliza
def get_Jacobi_method (A, tol = 1e-2, max_it = 100):
    
    n = A.shape[0]
    it = 0
    v = np.eye(n)
    
    while it < max_it:
        i, j, a = get_greatest_element(A)
        
        if a < tol:
            break
        
        if A[i, i] == A[j, j]:
            t = np.pi / 4
        else:
            t = 0.5 * np.arctan(2 * A[i, j] / (A[i, i] - A[j, j]))
            
        R_ij = R(i, j, t, n)
        
        A = R_ij.T @ A @ R_ij
        v = v @ R_ij
        
        it += 1
        
    return A, v
    
    
        
        

In [28]:
A = np.array([[4, 1, 1], 
              [1, 3, 2], 
              [1, 2, 5]])
print('valores propios metodo Jacobi: ')
print(np.linalg.diagonal(get_Jacobi_method(A)[0]))
print('vectores propios metodo Jacobi: ',get_Jacobi_method(A)[1])
print(np.linalg.eig(A))


valores propios metodo Jacobi: 
[3.39729482 1.70759867 6.89510652]
vectores propios metodo Jacobi:  [[ 0.8856695  -0.17094212  0.43170398]
 [-0.07555865  0.86430837  0.49725438]
 [-0.45812708 -0.47302201  0.75257541]]
EigResult(eigenvalues=array([6.89510652, 3.39729507, 1.70759841]), eigenvectors=array([[ 0.43170413,  0.88573564,  0.17059871],
       [ 0.49725362, -0.07589338, -0.86427949],
       [ 0.75257583, -0.45794385,  0.47319874]]))


In [7]:
#12. Metodo de Newton-Raphson y descenso de gradiente:

# FUNCIONES 
F1 = (lambda x,y: np.log(x**2 + y**2) - np.sin(x*y) - np.log(2) - np.log(np.pi), \
      lambda x,y: np.exp(x - y) + np.cos(x*y))

X1 = np.array([2,2])

F2 = (lambda x,y,z: 6*x - 2*np.cos(y*z) - 1., \
      lambda x,y,z: 9*y + np.sqrt(x**2 + np.sin(z) + 1.06) + 0.9,\
      lambda x,y,z: 60*z + 3*np.exp(-x*y) + 10*np.pi - 3)

X2 = np.array([0,0,0])

#3D
def GetF(G,r):

  n = r.shape[0]
  v = np.zeros(n)

  for i in range(n):
    v[i] = G[i](r[0],r[1],r[2])

  return v

def Metric(G,r):
  return 0.5*np.linalg.norm(GetF(G,r))**2

def GetJacobian(f,r,h=1e-3):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf = r.copy()
      rb = r.copy()

      rf[j] = rf[j] + h
      rb[j] = rb[j] - h

      J[i,j] = (f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2]))/(2*h)

  return J

def Minimizer(G,r,lr=1e-5,epochs=int(1e2),error=1e-8):

  metric = 1
  it = 0

  while metric > error:

    J = GetJacobian(G,r).T
    V = GetF(G,r)

    r = r - lr*np.dot(J,V)

    metric = Metric(G,r)

    #print(r, V, metric)
    it += 1

  return r,it

#2D
def GetF2(G,r):

  n = r.shape[0]
  v = np.zeros(n)

  for i in range(n):
    v[i] = G[i](r[0],r[1])

  return v

def Metric2(G,r):
  return 0.5*np.linalg.norm(GetF2(G,r))**2

def GetJacobian2(f,r,h=1e-3):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf = r.copy()
      rb = r.copy()

      rf[j] = rf[j] + h
      rb[j] = rb[j] - h

      J[i,j] = (f[i](rf[0],rf[1]) - f[i](rb[0],rb[1]))/(2*h)

  return J

def Minimizer2(G,r,lr=0.001,epochs=int(1e2),error=1e-8):

  metric = 1
  it = 0

  while metric > error:

    J = GetJacobian2(G,r).T
    V = GetF2(G,r)

    r = r - lr*np.dot(J,V)

    metric = Metric2(G,r)

    #print(r, V, metric)
    it += 1

  return r,it

print('Soluciones sistema 1: ', Minimizer2(F1,X1)[0],' en ',Minimizer2(F1,X1)[1],' iteraciones.')
print('Soluciones sistema 2: ', Minimizer(F2,X2)[0],' en ',Minimizer(F2,X2)[1],' iteraciones.')

Soluciones sistema 1:  [1.77238317 1.77252454]  en  5045  iteraciones.
Soluciones sistema 2:  [ 0.49812119 -0.19960367 -0.52882566]  en  27769  iteraciones.


13. Sea el operador derivada central de orden $\mathcal{O}(h^4)$:
$$
\frac{df}{dx}=\frac{-f(x+2h)+8f(x+h)-8f(x-h)+f(x-2h)}{12h}
$$

In [8]:
#a)
def GetJacobian2(f,r,h=0.01):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf1 = r.copy()
      rb1 = r.copy()
      rf2 = r.copy()
      rb2 = r.copy()

      rf1[j] = rf1[j] + h
      rb1[j] = rb1[j] - h
      
      rf2[j] = rf2[j] + (2*h)
      rb2[j] = rb2[j] - (2*h)

      J[i,j] = ((-f[i](rf2[0],rf2[1],rf2[2]))+(8*f[i](rf1[0],rf1[1],rf1[2]))-(8*f[i](rb1[0],rb1[1],rb1[2]))+(f[i](rb2[0],rb2[1],rb2[2])))/(12*h)

  return J

#b)
print('Jacobiano de orden O(h4): ', GetJacobian2(F2, np.array([0.5,0.5,0.5])))

#c)
def GetJacobian(f,r,h=0.01):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf = r.copy()
      rb = r.copy()

      rf[j] = rf[j] + h
      rb[j] = rb[j] - h

      J[i,j] = (f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2]))/(2*h)

  return J
print('Jacobiano de orden O(h2): ', GetJacobian(F2, np.array([0.5,0.5,0.5])))


Jacobiano de orden O(h4):  [[ 6.          0.24740396  0.24740396]
 [ 0.37377753  9.          0.32802064]
 [-1.16820117 -1.16820117 60.        ]]
Jacobiano de orden O(h2):  [[ 6.          0.24740293  0.24740293]
 [ 0.37376854  9.          0.32801836]
 [-1.16820604 -1.16820604 60.        ]]


En la derivada es de orden $\mathcal{O}(h^4)$, esto implica que error entre el error asociado al jacobiano generado y el jacobiano exacto debe ser del orden de $h^4$ para cada uno de sus elementos. Por otro lado, el error asociado a la derivada con $\mathcal{O}(h^2)$ es del orden de $h^2$. Por lo que, en teoria, si se elige un h tal que pueda aproximar el orden de error de $\mathcal{O}(h^4)$, ambos jacobianos serian los mismos.

In [9]:
print('Jacobiano de orden O(h2) con h = 1e-4: ', GetJacobian(F2, np.array([0.5,0.5,0.5]), h = 1e-4))

Jacobiano de orden O(h2) con h = 1e-4:  [[ 6.          0.24740396  0.24740396]
 [ 0.37377753  9.          0.32802064]
 [-1.16820118 -1.16820118 60.        ]]


In [10]:
#14)

#Matrices:
Jx = np.array([[0, 0, 0], 
               [0, 0, -1], 
               [0, 1, 0]])
Jy = np.array([[0, 0, 1], 
               [0, 0, 0], 
               [-1, 0, 0]])
Jz = np.array([[0, -1, 0], 
               [1, 0, 0], 
               [0, 0, 0]])

#Donde x,y,z son 1,2,3 respectivamente

#Conmutador
def conmutador (A, B):
    return (A@B) - (B@A)

def levi_civita(i, j, k):
    indices = (i,j,k)
    if (indices == (1,2,3)) or (indices == (2,3,1)) or (indices == (3,1,2)):
        return 1
    elif (indices == (3,2,1)) or (indices == (1,3,2)) or (indices == (2,1,3)):
        return -1
    else:
        return 0
    
#Matrices Jx y Jy
print('Algebra de Lie: Matrices Jx y Jy \n')
print('Conmutador: \n', conmutador(Jx, Jy))
print('Simbolo de Levi-Civita: \n', levi_civita(1, 2, 3)*Jz)

#Matrices Jx y Jz
print('Algebra de Lie: Matrices Jx y Jz \n')
print('Conmutador: \n', conmutador(Jx, Jz))
print('Simbolo de Levi-Civita: \n', levi_civita(1, 3, 2)*Jy)

#Matrices Jy y Jx
print('Algebra de Lie: Matrices Jy y Jx \n')
print('Conmutador: \n', conmutador(Jy, Jx))
print('Simbolo de Levi-Civita: \n', levi_civita(2, 1, 3)*Jz)
#Matrices Jy y Jz
print('Algebra de Lie: Matrices Jy y Jz \n')
print('Conmutador: \n', conmutador(Jy, Jz))
print('Simbolo de Levi-Civita: \n', levi_civita(2, 3, 1)*Jx)

#Matrices Jz y Jx
print('Algebra de Lie: Matrices Jz y Jx \n')
print('Conmutador: \n', conmutador(Jz, Jx))
print('Simbolo de Levi-Civita: \n', levi_civita(3, 1, 2)*Jy)
#Matrices Jz y Jy
print('Algebra de Lie: Matrices Jz y Jy \n')
print('Conmutador: \n', conmutador(Jz, Jy))
print('Simbolo de Levi-Civita: \n', levi_civita(3, 2, 1)*Jx)


Algebra de Lie: Matrices Jx y Jy 

Conmutador: 
 [[ 0 -1  0]
 [ 1  0  0]
 [ 0  0  0]]
Simbolo de Levi-Civita: 
 [[ 0 -1  0]
 [ 1  0  0]
 [ 0  0  0]]
Algebra de Lie: Matrices Jx y Jz 

Conmutador: 
 [[ 0  0 -1]
 [ 0  0  0]
 [ 1  0  0]]
Simbolo de Levi-Civita: 
 [[ 0  0 -1]
 [ 0  0  0]
 [ 1  0  0]]
Algebra de Lie: Matrices Jy y Jx 

Conmutador: 
 [[ 0  1  0]
 [-1  0  0]
 [ 0  0  0]]
Simbolo de Levi-Civita: 
 [[ 0  1  0]
 [-1  0  0]
 [ 0  0  0]]
Algebra de Lie: Matrices Jy y Jz 

Conmutador: 
 [[ 0  0  0]
 [ 0  0 -1]
 [ 0  1  0]]
Simbolo de Levi-Civita: 
 [[ 0  0  0]
 [ 0  0 -1]
 [ 0  1  0]]
Algebra de Lie: Matrices Jz y Jx 

Conmutador: 
 [[ 0  0  1]
 [ 0  0  0]
 [-1  0  0]]
Simbolo de Levi-Civita: 
 [[ 0  0  1]
 [ 0  0  0]
 [-1  0  0]]
Algebra de Lie: Matrices Jz y Jy 

Conmutador: 
 [[ 0  0  0]
 [ 0  0  1]
 [ 0 -1  0]]
Simbolo de Levi-Civita: 
 [[ 0  0  0]
 [ 0  0  1]
 [ 0 -1  0]]
